In [32]:
import pandas as pd

In [33]:
df = pd.read_csv("world_population.csv")

In [34]:
df.head()
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Rank                         234 non-null    int64  
 1   CCA3                         234 non-null    object 
 2   Country/Territory            234 non-null    object 
 3   Capital                      234 non-null    object 
 4   Continent                    234 non-null    object 
 5   2022 Population              234 non-null    int64  
 6   2020 Population              234 non-null    int64  
 7   2015 Population              234 non-null    int64  
 8   2010 Population              234 non-null    int64  
 9   2000 Population              234 non-null    int64  
 10  1990 Population              234 non-null    int64  
 11  1980 Population              234 non-null    int64  
 12  1970 Population              234 non-null    int64  
 13  Area (km²)          

,Rank,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
count,234.000000,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,2.340000e+02,234.000000,234.000000,234.000000
mean,117.500000,3.407441e+07,3.350107e+07,3.172996e+07,2.984524e+07,2.626947e+07,2.271022e+07,1.898462e+07,1.578691e+07,5.814494e+05,452.127044,1.009577,0.427051
std,67.694165,1.367664e+08,1.355899e+08,1.304050e+08,1.242185e+08,1.116982e+08,9.783217e+07,8.178519e+07,6.779509e+07,1.761841e+06,2066.121904,0.013385,1.714977
min,1.000000,5.100000e+02,5.200000e+02,5.640000e+02,5.960000e+02,6.510000e+02,7.000000e+02,7.330000e+02,7.520000e+02,1.000000e+00,0.026100,0.912000,0.000000
25%,59.250000,4.197385e+05,4.152845e+05,4.046760e+05,3.931490e+05,3.272420e+05,2.641158e+05,2.296142e+05,1.559970e+05,2.650000e+03,38.417875,1.001775,0.010000
50%,117.500000,5.559944e+06,5.493074e+06,5.307400e+06,4.942770e+06,4.292907e+06,3.825410e+06,3.141146e+06,2.604830e+06,8.119950e+04,95.346750,1.007900,0.070000
75%,175.750000,2.247650e+07,2.144798e+07,1.973085e+07,1.915957e+07,1.576230e+07,1.186923e+07,9.826054e+06,8.817329e+06,4.304258e+05,238.933250,1.016950,0.280000
max,234.000000,1.425887e+09,1.424930e+09,1.393715e+09,1.348191e+09,1.264099e+09,1.153704e+09,9.823725e+08,8.225344e+08,1.709824e+07,23172.266700,1.069100,17.880000


In [35]:
target = "2022 Population"

In [36]:
X = df.drop(columns=[target])
y = df[target]

In [37]:
numeric_cols = X.select_dtypes(include='number').columns

In [38]:
categorical_cols = X.select_dtypes(exclude='number').columns

In [39]:
numeric_cols.tolist()
categorical_cols.tolist()

['CCA3', 'Country/Territory', 'Capital', 'Continent']

In [42]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
numeric_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

In [45]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
categorical_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('encoder', OneHotEncoder(handle_unknown='ignore'))])

In [47]:
from sklearn.compose import ColumnTransformer

In [ ]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transform, numeric_cols),('cat', categorical_transform, categorical_cols)])

In [49]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = Pipeline(steps=[('preprocess', preprocessor),('rf', RandomForestRegressor(n_estimators=200, random_state=42))])

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [54]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Rank', '2020 Population', '2015 Population', '2010 Population',
       '2000 Population', '1990 Population', '1980 Population',
       '1970 Population', 'Area (km²)', 'Density (per km²)', 'Growth Rate',
       'World Population Percentage'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['CCA3', 'Country/Territory', 'Capital', 'Continent'], dtype='object'))])),
                ('rf',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [55]:
y_pred = model.predict(X_test)

In [56]:
from sklearn.metrics import r2_score, mean_absolute_error

In [57]:
r2_score(y_test, y_pred)
mean_absolute_error(y_test, y_pred)

1887829.8370212764

In [59]:
sample = X.sample(1)
sample

,Rank,CCA3,Country/Territory,Capital,Continent,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
11,55,AUS,Australia,Canberra,Oceania,25670051,23820236,22019168,19017963,17048003,14706322,12595034,7692024,3.4032,1.0099,0.33


In [61]:
prediction = model.predict(sample)
int(prediction[0])

34498863